# 3.7 学習と検証の実施

- 本ファイルでは、PSPNetの学習と検証の実施を行います。AWSのGPUマシンで計算します。
- p2.xlargeで約12時間かかります。


# 学習目標

1.	PSPNetの学習と検証を実装できるようになる
2.	セマンティックセグメンテーションのファインチューニングを理解する


# 事前準備

- 本書に従い学習済みモデルのファイル「pspnet50_ADE20K.pth」をダウンロードし、フォルダ「weights」に用意します。

In [ ]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DataLoader作成

In [ ]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

# ファイルパスリスト作成
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

# Dataset作成
# (RGB)の色の平均値と標準偏差
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

# DataLoader作成
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# ネットワークモデル作成

In [ ]:
from utils.pspnet import PSPNet

# ファインチューニングでPSPNetを作成
# ADE20Kデータセットの学習済みモデルを使用、ADE20Kはクラス数が150です
net = PSPNet(n_classes=150)

# ADE20K学習済みパラメータをロード
state_dict = torch.load("./weights/pspnet50_ADE20K.pth")
net.load_state_dict(state_dict)

# 分類用の畳み込み層を、出力数21のものにつけかえる
n_classes = 21
net.decode_feature.classification = nn.Conv2d(
    in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

net.aux.classification = nn.Conv2d(
    in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

# 付け替えた畳み込み層を初期化する。活性化関数がシグモイド関数なのでXavierを使用する。


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)


net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)


print('ネットワーク設定完了：学習済みの重みをロードしました')


ネットワーク設定完了：学習済みの重みをロードしました


In [ ]:
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNeckPSP(
      (cb

# 損失関数を定義

In [ ]:
# 損失関数の設定
class PSPLoss(nn.Module):
    """PSPNetの損失関数のクラスです。"""

    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight  # aux_lossの重み

    def forward(self, outputs, targets):
        """
        損失関数の計算。

        Parameters
        ----------
        outputs : PSPNetの出力(tuple)
            (output=torch.Size([num_batch, 21, 475, 475]), output_aux=torch.Size([num_batch, 21, 475, 475]))。

        targets : [num_batch, 475, 475]
            正解のアノテーション情報

        Returns
        -------
        loss : テンソル
            損失の値
        """

        loss = F.cross_entropy(outputs[0], targets, reduction='mean')
        loss_aux = F.cross_entropy(outputs[1], targets, reduction='mean')

        return loss+self.aux_weight*loss_aux


criterion = PSPLoss(aux_weight=0.4)


# 最適化手法を設定

In [ ]:
# ファインチューニングなので、学習率は小さく
optimizer = optim.SGD([
    {'params': net.feature_conv.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
    {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
    {'params': net.decode_feature.parameters(), 'lr': 1e-2},
    {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)


# スケジューラーの設定
def lambda_epoch(epoch):
    max_epoch = 30
    return math.pow((1-epoch/max_epoch), 0.9)


scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)


# 学習・検証を実施する

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # 画像の枚数
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    num_val_imgs = len(dataloaders_dict["val"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # イテレーションカウンタをセット
    iteration = 1
    logs = []

    # multiple minibatch
    batch_multiplier = 3

    # epochのループ
    for epoch in range(num_epochs):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                scheduler.step()  # 最適化schedulerの更新
                optimizer.zero_grad()
                print('（train）')

            else:
                if((epoch+1) % 5 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は5回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            count = 0  # multiple minibatch
            for imges, anno_class_imges in dataloaders_dict[phase]:
                # ミニバッチがサイズが1だと、バッチノーマライゼーションでエラーになるのでさける
                if imges.size()[0] == 1:
                    continue

                # GPUが使えるならGPUにデータを送る
                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)

                
                # multiple minibatchでのパラメータの更新
                if (phase == 'train') and (count == 0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(imges)
                    loss = criterion(
                        outputs, anno_class_imges.long()) / batch_multiplier

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算
                        count -= 1  # multiple minibatch

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item() * batch_multiplier
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item() * batch_multiplier

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss /
                     num_train_imgs, 'val_loss': epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

    # 最後のネットワークを保存する
    torch.save(net.state_dict(), 'weights/pspnet50_' +
               str(epoch+1) + '.pth')


In [ ]:
# 学習・検証を実行する
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-------------
Epoch 1/30
-------------
（train）
イテレーション 10 || Loss: 0.3835 || 10iter: 83.2019 sec.
イテレーション 20 || Loss: 0.2189 || 10iter: 50.9118 sec.
イテレーション 30 || Loss: 0.1510 || 10iter: 50.8032 sec.
イテレーション 40 || Loss: 0.1658 || 10iter: 50.7695 sec.
イテレーション 50 || Loss: 0.0886 || 10iter: 50.6645 sec.
イテレーション 60 || Loss: 0.0728 || 10iter: 50.6198 sec.
イテレーション 70 || Loss: 0.1165 || 10iter: 50.9016 sec.
イテレーション 80 || Loss: 0.1351 || 10iter: 50.4392 sec.
イテレーション 90 || Loss: 0.2174 || 10iter: 50.6154 sec.
イテレーション 100 || Loss: 0.0904 || 10iter: 50.5267 sec.
イテレーション 110 || Loss: 0.1408 || 10iter: 50.4316 sec.
イテレーション 120 || Loss: 0.0668 || 10iter: 50.5083 sec.
イテレーション 130 || Loss: 0.1251 || 10iter: 50.7642 sec.
イテレーション 140 || Loss: 0.1467 || 10iter: 50.5163 sec.
イテレーション 150 || Loss: 0.0794 || 10iter: 50.5443 sec.
イテレーション 160 || Loss: 0.1435 || 10iter: 50.4825 sec.
イテレーション 170 || Loss: 0.2098 || 10iter: 50.4970 sec.
イテレーション 180 || Loss: 0.1435 || 10iter: 50.5844 sec.
-----------

イテレーション 1380 || Loss: 0.0715 || 10iter: 50.8677 sec.
イテレーション 1390 || Loss: 0.0787 || 10iter: 50.8265 sec.
イテレーション 1400 || Loss: 0.0590 || 10iter: 50.7918 sec.
イテレーション 1410 || Loss: 0.0693 || 10iter: 50.6000 sec.
イテレーション 1420 || Loss: 0.0749 || 10iter: 50.6794 sec.
イテレーション 1430 || Loss: 0.1086 || 10iter: 50.6710 sec.
イテレーション 1440 || Loss: 0.0649 || 10iter: 50.8908 sec.
イテレーション 1450 || Loss: 0.0552 || 10iter: 50.7711 sec.
イテレーション 1460 || Loss: 0.0374 || 10iter: 50.5102 sec.
-------------
epoch 8 || Epoch_TRAIN_Loss:0.0574 ||Epoch_VAL_Loss:0.0000
timer:  1026.2220 sec.
-------------
Epoch 9/30
-------------
（train）
イテレーション 1470 || Loss: 0.0475 || 10iter: 28.4563 sec.
イテレーション 1480 || Loss: 0.0209 || 10iter: 50.8128 sec.
イテレーション 1490 || Loss: 0.0394 || 10iter: 50.5409 sec.
イテレーション 1500 || Loss: 0.0464 || 10iter: 50.7096 sec.
イテレーション 1510 || Loss: 0.0371 || 10iter: 50.8713 sec.
イテレーション 1520 || Loss: 0.0364 || 10iter: 50.9948 sec.
イテレーション 1530 || Loss: 0.1000 || 10iter: 50.6096 sec.
イテレーション 1

イテレーション 2740 || Loss: 0.0271 || 10iter: 50.7591 sec.
-------------
（val）
-------------
epoch 15 || Epoch_TRAIN_Loss:0.0459 ||Epoch_VAL_Loss:0.0714
timer:  1347.2690 sec.
-------------
Epoch 16/30
-------------
（train）
イテレーション 2750 || Loss: 0.0726 || 10iter: 22.7602 sec.
イテレーション 2760 || Loss: 0.0297 || 10iter: 50.7317 sec.
イテレーション 2770 || Loss: 0.0294 || 10iter: 50.7034 sec.
イテレーション 2780 || Loss: 0.0386 || 10iter: 50.9082 sec.
イテレーション 2790 || Loss: 0.0381 || 10iter: 50.8296 sec.
イテレーション 2800 || Loss: 0.0466 || 10iter: 50.6776 sec.
イテレーション 2810 || Loss: 0.0318 || 10iter: 50.8160 sec.
イテレーション 2820 || Loss: 0.0314 || 10iter: 50.5198 sec.
イテレーション 2830 || Loss: 0.0482 || 10iter: 50.7523 sec.
イテレーション 2840 || Loss: 0.0258 || 10iter: 50.8890 sec.
イテレーション 2850 || Loss: 0.0436 || 10iter: 50.7661 sec.
イテレーション 2860 || Loss: 0.0503 || 10iter: 50.5520 sec.
イテレーション 2870 || Loss: 0.0293 || 10iter: 50.5255 sec.
イテレーション 2880 || Loss: 0.0287 || 10iter: 50.4744 sec.
イテレーション 2890 || Loss: 0.0360 || 10iter: 

イテレーション 4060 || Loss: 0.0297 || 10iter: 50.8080 sec.
イテレーション 4070 || Loss: 0.0244 || 10iter: 50.4723 sec.
イテレーション 4080 || Loss: 0.0669 || 10iter: 50.6491 sec.
イテレーション 4090 || Loss: 0.0236 || 10iter: 50.5933 sec.
イテレーション 4100 || Loss: 0.0330 || 10iter: 50.7665 sec.
イテレーション 4110 || Loss: 0.0284 || 10iter: 50.5859 sec.
イテレーション 4120 || Loss: 0.0409 || 10iter: 50.5714 sec.
イテレーション 4130 || Loss: 0.0505 || 10iter: 50.9581 sec.
イテレーション 4140 || Loss: 0.0372 || 10iter: 50.7392 sec.
イテレーション 4150 || Loss: 0.0457 || 10iter: 50.8464 sec.
イテレーション 4160 || Loss: 0.0450 || 10iter: 50.8673 sec.
イテレーション 4170 || Loss: 0.0761 || 10iter: 50.6163 sec.
イテレーション 4180 || Loss: 0.0455 || 10iter: 50.8787 sec.
イテレーション 4190 || Loss: 0.0393 || 10iter: 50.5992 sec.
イテレーション 4200 || Loss: 0.0424 || 10iter: 50.7738 sec.
-------------
epoch 23 || Epoch_TRAIN_Loss:0.0423 ||Epoch_VAL_Loss:0.0000
timer:  1025.3780 sec.
-------------
Epoch 24/30
-------------
（train）
イテレーション 4210 || Loss: 0.0193 || 10iter: 0.2901 sec.
イテレーション 

イテレーション 5420 || Loss: 0.0194 || 10iter: 50.6732 sec.
イテレーション 5430 || Loss: 0.0428 || 10iter: 50.8644 sec.
イテレーション 5440 || Loss: 0.0494 || 10iter: 50.8474 sec.
イテレーション 5450 || Loss: 0.0507 || 10iter: 50.8454 sec.
イテレーション 5460 || Loss: 0.0448 || 10iter: 51.0749 sec.
イテレーション 5470 || Loss: 0.0192 || 10iter: 51.0534 sec.
イテレーション 5480 || Loss: 0.0663 || 10iter: 50.8508 sec.
イテレーション 5490 || Loss: 0.0642 || 10iter: 50.8363 sec.
-------------
（val）
-------------
epoch 30 || Epoch_TRAIN_Loss:0.0399 ||Epoch_VAL_Loss:0.0701
timer:  1352.9696 sec.


以上